In [17]:
import ipywidgets as widgets
import pandas as pd
import numpy as np

import pymove as pm
from pymove.visualization import folium as f, matplotlib as mpl
from pymove.utils import visual

import pandas as pd
import pymove as pm
from pymove import folium, MoveDataFrame
from pymove.query import query

import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
move_df = pd.read_csv("atlantic.csv", encoding= 'unicode_escape')

In [19]:
move_df = pm.conversions.lat_and_lon_decimal_degrees_to_decimal(
    move_df, latitude='Latitude', longitude='Longitude'
)

def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
move_df['Datetime'] = move_df.apply(convert_to_datetime, axis=1)

#move_df[['ID', 'Name', 'Latitude', 'Longitude', 'Datetime']].head()

In [20]:
#Converting the pandas dataframe to pymove's MoveDataFrame
move_pm = MoveDataFrame(
    data=move_df, latitude='Latitude', longitude='Longitude',datetime='Datetime', traj_id='Name'
)
#print(type(move_pm))
#move_pm.head()

In [21]:
#move_df

In [22]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [23]:
def colour_ge_value(value, comparison):
    if value > comparison:
        return 'color: red'
    else:
        return 'color: black'

In [36]:
output = widgets.Output()
plot_output = widgets.Output()

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(move_df.ID), description = 'ID:')
dropdown_purpose = widgets.Dropdown(options = unique_sorted_values_plus_ALL(move_df.Time), description = 'Time:')
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1, description = 'Number:')


def common_filtering(Id, time, num):
    output.clear_output()
    plot_output.clear_output()
    
    if (Id == ALL) & (time == ALL):
        common_filter = move_df
    elif (Id == ALL):
        common_filter = move_df[move_df.Time == time]
    elif (time == ALL):
        common_filter = move_df[move_df.ID == Id]
    else:
        common_filter = move_df[(move_df.ID == Id) & (move_df.Time == time)]
    
    with output:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num), subset=['Latitude', 'Time']))
    with plot_output:
        sns.kdeplot(common_filter['Latitude'], shade=True)
        plt.show()
        

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value,
                     bounded_num.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, 
                     bounded_num.value)
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)

    

dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

#display(dropdown_year)
#display(dropdown_purpose)
#display(bounded_num)
#display(output)
#display(plot_output)

In [37]:
input_widgets = widgets.HBox(
[dropdown_year, dropdown_purpose, bounded_num])
display(input_widgets)

tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')
display(tab)